In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [426]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0','Unit1','Unit2'])

In [427]:
patient_df[patient_df.pid=="p00001"].head()
patient_df.shape
patient_df.HospAdmTime.unique().shape
patient_df[patient_df.SepsisLabel == 1].HospAdmTime.unique()

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,1,0
179973,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,2,0
179974,p00001,91.0,96.0,36.5,108.0,84.5,72.0,23.5,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,3,0
179975,p00001,93.0,98.0,36.5,123.0,87.0,61.0,21.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,4,0
179976,p00001,93.0,95.0,36.5,110.0,81.0,70.0,20.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,5,0


(188453, 40)

(2976,)

array([ 0.00000e+00, -3.10000e-01, -5.39000e+01, -4.91000e+00,
       -5.00000e-02, -2.14600e+01, -1.32730e+02, -3.00470e+02,
       -1.92270e+02, -3.04000e+01, -8.01000e+00, -5.28000e+00,
       -2.43200e+01, -2.85800e+01, -4.91800e+01, -5.93740e+02,
       -2.81780e+02, -4.28000e+00, -3.53000e+01, -9.93000e+00,
       -1.20000e-01, -2.00000e-02, -5.63820e+02, -3.50260e+02,
       -3.19850e+02, -9.21000e+00, -6.03400e+01, -5.27240e+02,
       -2.47000e+00, -1.17172e+03, -7.52000e+00, -3.08740e+02,
       -1.15000e+00, -3.00000e-02, -2.60000e+00, -7.00000e-02,
       -2.52300e+01, -1.00000e-02, -7.76200e+01, -1.97780e+02,
       -7.08400e+01, -7.29000e+00, -2.58200e+01, -3.31000e+00,
       -2.66400e+01, -3.58800e+01, -9.75700e+01, -2.51100e+01,
       -2.92000e+00, -2.34000e+00, -1.01100e+01, -1.10000e-01,
       -1.19900e+01, -4.40000e-01, -9.07000e+00, -1.70000e-01,
       -2.69600e+01, -3.64940e+02, -7.21200e+01, -4.95000e+00,
       -2.91000e+00, -4.00000e-02, -8.00000e-02, -3.300

In [130]:
#patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

In [428]:
patient_df[patient_df.pid=="p00001"].SepsisLabel.unique()

array([0], dtype=int64)

In [429]:
patient_df.pid.unique().shape

(5000,)

In [437]:
re_idx = patient_df.set_index(["pid","ICULOS"])
arr = re_idx.groupby("pid").apply(lambda x: x.values).values

train_raw, test_raw = train_test_split(arr, test_size=0.2)

In [438]:
train_raw[0].shape
train_raw[0][0]

(14, 38)

array([  74.5       ,  100.        ,   36.45      ,  110.5       ,
         84.        ,   69.5       ,   16.        ,   18.5       ,
         -3.16091766,   22.72753865,    0.8       ,    7.4       ,
         40.        ,   96.5030266 ,  116.31360923,    8.        ,
         91.34764342,    1.11      ,  105.        ,    0.86      ,
          0.85336036,  131.        ,    1.47      ,    2.3       ,
          4.1       ,    4.2       ,    1.44430028,    5.67129436,
         25.7       ,    8.        ,   40.80889022,   14.3       ,
        296.93274106,  316.        ,   50.        ,    1.        ,
       -102.04      ,    0.        ])

In [439]:
# create windowing system here
T = 6
#idx = 10
def process_data(df: pd.DataFrame, T: int) -> pd.DataFrame:
    processed = []

    for idx in range(df.shape[0]):
        if df[idx][:,-1].sum() == 0:
            processed.extend([[row,7,1] for row in df[idx][:,:-1]])
        else:
            sepsis_count = 0
            for i in range(df[idx].shape[0]):
                t = (T + 1) - sepsis_count
                t = t if t >= 1 else 1
                s = 1 if t > T else 0
                processed.append([df[idx][i][:-1],t,s])
                sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
    return pd.DataFrame(processed, columns=["x","t","s"])
# Naive windowing:
#             for i in range(df[idx].shape[0]):
#                 window = df[idx][i:i+T]
#                 matches = np.where(window[:,-1]==1)[0]
#                 if matches.size > 0:
#                     t = matches[0] + 1
#                     s = 0
#                 else:
#                     t = T + 1
#                     s = 1
#                 processed.append([df[idx][i][:-1],t,s])

# def test(df: pd.DataFrame, T: int) -> pd.DataFrame:
#     processed = []

#     #for idx in range(df.shape[0]):
#     idx = 10
#     if df[idx][:,-1].sum() == 0:
#         processed.extend([[row,7,1] for row in df[idx][:,:-1]])
#     else:
#         sepsis_count = 0
#         for i in range(df[idx].shape[0]):
#             t = (T + 1) - sepsis_count
#             t = t if t >= 1 else 1
#             s = 1 if t > T else 0
#             processed.append([df[idx][i][:-1],t,s])
#             sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
#     return pd.DataFrame(processed, columns=["x","t","s"])

# np.array(train[idx][:,:-1]).shape
# np.repeat([[7,1]],train[idx].shape[0],axis=0).shape
# t_s = np.repeat([[7,1]],train[idx].shape[0],axis=0)
# t_s
# x_idx = train[idx][:,:-1].reshape(train[idx][:,:-1].shape[0],1,-1)
# x_idx.shape
# processed_idx = np.concatenate((x_idx,t_s),axis=1)
# processed_idx.shape
#if train[idx][:,-1].sum() == 0:
# a = np.ones((4,1)) * 3
# b = np.repeat([[7,1]],4,axis=0)    
# a
# b
# np.hstack((a,b))
#pd.DataFrame([[train[idx][i][:-1],t,s]], columns=["x","t","s"])
# for idx in range(4000):
#     #window = train[idx][i:i+T]
#     if train[idx][:,-1].sum() > 0:
#         break

In [440]:
train = process_data(train_raw, T)
test = process_data(test_raw, T)

In [441]:
df[340:400]

,x,t,s
340,"[88.0, 100.0, 37.3, 137.0, 104.0, 82.0, 12.0, ...",7,1
341,"[88.0, 99.0, 37.3, 130.0, 97.0, 74.0, 14.0, 34...",7,1
342,"[87.0, 100.0, 37.3, 125.0, 97.0, 80.0, 12.0, 3...",7,1
343,"[87.0, 100.0, 37.3, 158.0, 97.0, 68.0, 12.0, 3...",7,1
344,"[93.0, 98.0, 36.7, 99.0, 76.0, 62.0, 12.0, 34....",7,1
345,"[79.0, 95.0, 36.7, 117.0, 89.0, 70.0, 21.0, 34...",7,1
346,"[73.0, 98.0, 36.7, 117.0, 95.0, 80.0, 20.0, 34...",7,1
347,"[87.0, 97.0, 36.7, 130.0, 96.0, 74.0, 22.0, 34...",7,1
348,"[76.0, 96.0, 36.7, 117.0, 89.0, 70.0, 23.0, 34...",7,1
349,"[77.0, 97.0, 36.7, 127.0, 97.0, 78.0, 20.0, 34...",7,1


In [436]:
df.head()

,x,t,s
0,"[80.0, 100.0, 37.0, 160.5, 87.5, 65.0, 25.0, 3...",7,1
1,"[80.0, 100.0, 37.0, 160.5, 87.5, 65.0, 25.0, 3...",7,1
2,"[84.0, 100.0, 37.0, 158.0, 95.0, 66.0, 25.0, 3...",7,1
3,"[83.0, 100.0, 37.0, 142.0, 92.0, 64.0, 22.0, 3...",7,1
4,"[82.0, 100.0, 37.0, 162.0, 116.0, 108.0, 22.0,...",7,1


In [87]:
df = pd.DataFrame(np.array([[12,1,1,1],[12,2,2,2],[12,3,2,2],[13,1,3,3],[13,2,4,4],[14,1,5,5],[14,2,6,6],[15,1,5,5]]), columns=["id","los","b","c"])
df.set_index(["id","los"], inplace=True)
df
arr = df.groupby("id").apply(lambda x: x.values).values
train, test = train_test_split(arr, test_size=0.2)
train
test

b  c
id los      
12 1    1  1
   2    2  2
   3    2  2
13 1    3  3
   2    4  4
14 1    5  5
   2    6  6
15 1    5  5

array([array([[5, 5],
       [6, 6]]),
       array([[1, 1],
       [2, 2],
       [2, 2]]),
       array([[3, 3],
       [4, 4]])], dtype=object)

array([array([[5, 5]])], dtype=object)